# Fine-tuning GPT2-medium on jokes dataset in PyTorch

This is an experimental notebook for fine-tuning pretrained GPT2-medium model on jokes dataset. Let's see if it can learn to crack some jokes. 

For this purpose I will use pretrained GPT2 medium size model from huggingface [transformers repository](https://github.com/huggingface/transformers).

#### First, check out the *GPT2LMHeadModel* text generation experiments in this [gist](www.com) . 

In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [3]:
# Function to first select topN tokens from the probability list and then based on the N word distribution
# select random token ID
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

### PyTorch Dataset module for Redit jokes

For fine-tuning the GPT2 model I will use [this](https://github.com/taivop/joke-dataset/blob/master/reddit_jokes.json) jokes dataset. After each joke string I add "<|endofext|>" which is recognized by the GPT2 model as and end of text marker. The marker will allow me to concatenate many jokes in one sequence.

In [8]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json

class JokesDataset(Dataset):
    def __init__(self, jokes_dataset_path = 'jokes_data/'):
        super().__init__()

        reddit_jokes_path = os.path.join(jokes_dataset_path, 'reddit_jokes.json')

        with open(reddit_jokes_path) as f:
            data = json.load(f)

        self.joke_list = []
        self.end_of_text_token = "<|endoftext|>"

        for idx, joke_json in enumerate(data):
            joke_str = f"{self.end_of_text_token}START:{joke_json['title']} {joke_json['body']}{self.end_of_text_token}"
            self.joke_list.append(joke_str)
            
    def __len__(self):
        return len(self.joke_list)

    def __getitem__(self, item):
        return self.joke_list[item]


In [9]:
dataset = JokesDataset()
joke_loader = DataLoader(dataset, batch_size=1, shuffle=True)

### Fine-tuning GPT2-medium on a single GPU
 
Large Transformer models are usually trained in multi-GPU(or TPU) setting because training on reasonable batch size and sequence length requires a lots of [tensor|graphical] processing unit memory. My machine is equipped with a single GeForce 1080 Ti which has 11 GB of memory. By empirical tests on GPT2 medium model I found that the maximum total sequence element count over all batches for my GPU to backprop trough is approximetly 550, which is not a lot and might not be sufficient for successful fine-tuning. 

But there are some things we can take into account and improve the situation. 

First thing to notice is that batch size in forward/ backward pass of Transformer does not play a role because [Layer Normalization](https://mlexplained.com/2018/11/30/an-overview-of-normalization-methods-in-deep-learning/) is used instead of Batch Normalization. In Layer Normalization each feature is normalized across the features dimension instead across the batch dimension as in the Batch Normalization. 

Second, we can collect gradients over multiple forward-backward passes and only then do the optimization step. This way we can store in the memory of the GPU a computational graph of one sequence at a time instead of storing coputational graph of all of the batch. With this strategy we can get the same result as if the batch would have been processed in a single forward/backward pass, only with *BATCH_SIZE* times less memory.

Putting it all together - I will process one sequence at a time with maximum length of 550. The length of joke sequences vary a lot in the dataset I use and to make the the total sequence element count in one optimization step more consistent I will try to fit in as many jokes as possible in the 550 element sequence. 

### Hyperparameters

I tested many(I think 5) hyperparameter sets till I found one that works the best. I mostly changed ***BATCH_SIZE*** (in this case it is the number of forward-backward passes between each optimization step), ***EOPOCHS***, and ***LEARNING_RATE***.

For a parameter value starting point for fine-tuning I inspired from [this](https://github.com/huggingface/transformers/blob/master/examples/run_squad.py) and [this](https://github.com/huggingface/transformers/blob/master/examples/run_glue.py).

In [11]:
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 3e-5
WARMUP_STEPS = 10000
MAX_SEQ_LEN = 400
from transformers import AdamW, WarmupLinearSchedule

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [12]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx,joke in enumerate(joke_loader):
        
        #################### "Fit as many joke sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        #Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if joke_tens.size()[1] > MAX_SEQ_LEN:
            continue
        
        #The first joke sequence in the sequence
        if not torch.is_tensor(tmp_jokes_tens):
            tmp_jokes_tens = joke_tens
            continue
        else:
            #The next joke does not fit in so we process the sequence and leave the last joke 
            #as the start for next sequence 
            if tmp_jokes_tens.size()[1] + joke_tens.size()[1] > MAX_SEQ_LEN:
                work_jokes_tens = tmp_jokes_tens
                tmp_jokes_tens = joke_tens
            else:
                #Add the joke to sequence, continue and try to add more
                tmp_jokes_tens = torch.cat([tmp_jokes_tens, joke_tens[:,1:]], dim=1)
                continue
        ################## Sequence ready, process it trough the model ##################
            
        outputs = model(work_jokes_tens, labels=work_jokes_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()
            
        if batch_count == 1000:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

EPOCH 0 started==============================
sum loss 3436.03076171875
sum loss 3271.525390625
sum loss 3117.43603515625
sum loss 3020.50732421875
sum loss 2940.70654296875
sum loss 2915.05078125
sum loss 2895.248779296875
sum loss 2849.1494140625
sum loss 2863.0771484375
sum loss 2827.261474609375
sum loss 2824.12109375
sum loss 2795.527587890625
sum loss 2803.104248046875
sum loss 2802.185791015625
sum loss 2786.28515625
sum loss 2778.56982421875
sum loss 2762.30615234375
sum loss 2770.957763671875
sum loss 2754.240478515625
sum loss 2747.343994140625
sum loss 2726.651611328125
sum loss 2729.69189453125
sum loss 2744.18212890625
sum loss 2724.10400390625
sum loss 2714.15283203125
sum loss 2711.803955078125
sum loss 2703.7119140625
sum loss 2704.89208984375
sum loss 2697.152099609375
sum loss 2689.19189453125
sum loss 2671.143798828125
sum loss 2677.786376953125
EPOCH 1 started==============================
sum loss 2645.66064453125
sum loss 2622.15966796875
sum loss 2662.0419921875


### And finally - generating some jokes

In [13]:
model.eval()
with torch.no_grad():
    
    for joke_idx in range(500):

        cur_ids = torch.tensor(tokenizer.encode("<|endoftext|>START:")).unsqueeze(0).to(device)
        
        for i in range(250):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one in this case) batch and the last predicted embedding
            if i < 2:
                n = 15
            else:
                n = 3
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

            if next_token_id in tokenizer.encode('<|endoftext|>'):
                break
            
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        print(f"JOKE NR {joke_idx}: {output_text} \n")

JOKE NR 0: <|endoftext|>START:What's the hardest to get off your face? The elevator. 

I'll show myself out.<|endoftext|> 

JOKE NR 1: <|endoftext|>START:A man is driving through the countryside when he sees a farmer in a field... He pulls over and asks the farmer, "Excuse me, but I noticed you're not wearing a seatbelt. What's the matter?" The farmer says, "I was just driving past the field and saw a cow grazing on a fence. It scared the bejesus out of me!" So the man pulls over to the side of the road and asks the farmer if he's okay. The farmer says "I'm fine, I was just driving past a field and saw a cow grazing on a fence. I thought it was a cow. I drove over to check and sure enough, it was a bull." The man says, "That's amazing, I was driving by the field when I saw a cow with her hind legs raised up in a sign of pride. I was so excited, I drove over to check and sure enough, it was a bull." The man says, "That's amazing, I was driving by the field when I saw a cow with its hind

JOKE NR 17: <|endoftext|>START:How do you get a girl pregnant? You take the tampon out of the box.<|endoftext|> 

JOKE NR 18: <|endoftext|>START:A man is walking through the forest when he sees an owl perched on a branch. The owl looks at him and says, 'I'm going to eat you.' 

'What's the harm?' the man asks. 

'I'm an American,' the owl replies. 'I can't eat a Canadian, so I'm going to eat you.' 

The man is taken aback, but he doesn't say anything and continues walking. 

A few minutes later, a rabbit is perched on a branch. 

'What's the harm?' the man asks the rabbit. 

'I'm an American,' the rabbit says. 'I can't eat a Canadian, so I'm going to eat you.' 

The rabbit is taken aback, but he doesn't say anything and continues walking. 

Finally, a bear is on the branch. 

'What's the matter with you, man?' the bear asks. 

The man replies, 'I'm an American, and I'm going to eat you.' 

'Why not?' the bear asks, 'I'm a Canadian too.' 

The man replies, 'I 

JOKE NR 19: <|endoftext|>

JOKE NR 36: <|endoftext|>START:Two men are walking through the woods when they come across a bear. One of their friend asks, "Hey, what are you doing?"

The other man replies, "We're hunters."<|endoftext|> 

JOKE NR 37: <|endoftext|>START:Why would you never trust a man with a broken pencil? He's a pencil thief.<|endoftext|> 

JOKE NR 38: <|endoftext|>START:A man is walking through a forest... A man is walking through the woods when he comes across a bear. He approaches it, and after a few minutes of staring at the bear, the bear stops.

"You're not going to eat me?" the bear asked.

The man replies, "No, you're not going to eat me. I have a gun in my pocket and a knife with me, and I don't want you to get any closer."

The bear then proceeds to climb up on top of the man, and the bear starts to eat the man. The bear then looks at the man, and says, "I'm going to give you the same punishment I gave the bear. You have to eat me."

The man, shocked by this, says, "What punishment? I didn

JOKE NR 55: <|endoftext|>START:What did the pirate say to the pirate? "I am the best!"<|endoftext|> 

JOKE NR 56: <|endoftext|>START:A blonde, a redhead and a brunette are all sitting in a bar. A blonde, a redhead and a brunette are all sitting in a bar. The blonde looks over to the brunette and says "You know what, you know I can take the whole bar with me. I'm a brunette and I can take a blonde with me."

The brunette says "I'll take you with me, too. I'm also a brunette and I can take a redhead." The redhead looks over at the blonde, and says "You know what, that's a no brainer. I'll take you with me. I'm a redhead too. I can take a blonde too." The brunette looks at him and says "Well, I'm a blonde too, but I can take a brunette with me."

The blonde looks at her friend and says "Well, you're right. If you want to go with me, go ahead. I'm a blonde, too. I'm a blonde too."

The blonde looks back and says "Well, I'll be damned. If I'm going to be a blond, I want to be a blond blonde

JOKE NR 78: <|endoftext|>START:The Perfect Son. A: I have the perfect son. 
B: Does he smoke? 
A: No, he doesn't. 
B: Does he drink whiskey? 
A: No, he doesn't. 
B: Does he ever come home late? 
A: No, he doesn't. 
B: I guess you really do have the perfect son. How old is he? 
A: He will be six months old next Wednesday.<|endoftext|> 

JOKE NR 79: <|endoftext|>START:An American, a Russian and a German are on a plane... An American, a Russian and a German are on a plane. The plane is going down and the pilot is yelling, “We’re going down. We can't get back up. We’ve only got two parachutes.” The American jumps and lands. He jumps again and lands. The German jumps and the plane crashes to the ground.

The pilot yells, “We have parachutes. Get up.” The parachutes are taken off and the pilot yells, “Get up, you stupid bastards! You’ve ruined our lives and I’ll nail your fucking feet to the floor!”<|endoftext|> 

JOKE NR 80: <|endoftext|>START:The man who invented the vacuum cleaner......wa

...

JOKE NR 415: <|endoftext|>START:Did you hear about the guy who was arrested for stealing a car? He was charged with car theft<|endoftext|> 

JOKE NR 416: <|endoftext|>START:A man goes to the doctor for a check up. Doctor : I have some bad news.  You have to stop masturbating.
Man : What's wrong?
Doctor : Well I'm afraid you've got cancer.
Man : What? What is it?
Doctor : Your testicles are cancerous.
Man : Wow that's awful, what can I do?
Doctor : You can stop masturbating and you can come back in a week.

Man comes back the week after and the Doctor says "Well, you have to stop masturbating, I'm afraid you have cancer.  What's the problem?"
Man : Doctor, I don't have cancer.
Doctor : Well I'm afraid you have AIDS.
Man : What?! What's AIDS?
Doctor : Your testicles are cancerous.
Man : Wow that's awful, what can I do?
Doctor : Go to the doctor, he'll give you the cure.<|endoftext|> 

JOKE NR 417: <|endoftext|>START:Why aren't there any black jokes on this subreddit? I'm not sure, I've h

JOKE NR 436: <|endoftext|>START:A woman is walking through a park......and she comes across a dog. The dog is obviously a pit bull, and the woman thinks to herself, "Wow, I wish I had this dog."

The woman goes up to the dog and says, "I'm going to give you a bone."

The dog says to her, "You're a real bitch, you're not supposed to give a bone to a pit bull. I'm going to bite you in the ass."

The woman is shocked. "Oh my god, I thought you said you were a pit bull."

The woman says, "Yeah. I'm going to bite your balls."

The dog says, "No, no. I'm going to bite your tits. You're not supposed to give a bone to a pit bull."

The woman says, "Oh my god, I thought you said you were a pit bull. I'm going to bite your tits!"

The dog says, "No, no. I'm gonna bite your tits!"

The woman says, "No, no, no, no, no, no, no, no, no, no, no, no, 

JOKE NR 437: <|endoftext|>START:A guy walks into a bar......and sits down at the bar. He orders three beers and starts drinking them. He finishes them 

JOKE NR 455: <|endoftext|>START:What did the pirate say to the pirate captain when they were in trouble? "I'll be a good boy and let's go back."<|endoftext|> 

JOKE NR 456: <|endoftext|>START:So, I went to the doctor......and he asked me if I had ever had sex before.

"No" I replied.

"Well, you have to stop masturbating."

"What?" I said. "I'm over here."

"You have to stop touching yourself."

"Oh, I'm over here too."

"You have to stop masturbating."

"What?" I asked.

"You have to stop touching yourself."

I said, "But, I'm over here."

The Doctor looked at me and said, "You have to stop masturbating, or I'll call the cops."

I said, "But, I'm over here too."

"Well, then, you have to stop touching yourself."

I looked at the doctor, then back to him and said, "But I'm over here too."

"You have to stop masturbating."

"What?"

"You have to stop masturbating."

"I can't stop masturbating, I'm over here too."

"You have to stop masturbating."

"What?
 

JOKE NR 457: <|endoftext|>STA

JOKE NR 472: <|endoftext|>START:An elderly couple were having dinner... An elderly couple were having dinner and they were having some conversation. 

The wife asks, "What's your favorite letter in the alphabet?" 

The husband replies, "I love the letter 'C' because it's the hardest letter in the world to pronounce."

The wife asks, "What's your favorite letter in the alphabet?"

The husband replies, "I love the letter 'D', because it's the most common letter in the world. 

The wife says, "Well, what's your favorite letter in the alphabet?"

The husband says, "The letter 'F'. 

The wife asks, "Well what's your favorite letter in the alphabet?" 

The husband responds, "The letter 'G', because it's the most common letter in the world."

The wife asks, "Well what's your favorite letter in the alphabet?"

The husband says, "The letter 'H'. 

The wife says, "What the hell, what's your favorite letter in the alphabet?"

The husband says, "The letter 'I'. 
The wife asks, "Well what is 

JOKE

JOKE NR 488: <|endoftext|>START:A man goes to the hospital to have his appendix removed. The doctor asks the man, "How long have you been having these?"

The man says, "About a month and a half."

"What happened?" asks the doctor.

"Well, my wife and I had a little fight and I had to go to the hospital. I was in the operating room, and I was in the middle of the operation, and I heard my wife screaming and crying, 'I'm pregnant! I'm pregnant!'"

The doctor asks, "What was she doing?"

The man says, "She was trying to get me to stop screaming."<|endoftext|> 

JOKE NR 489: <|endoftext|>START:A man goes into a bar... A man goes into a bar. He sits at a table and orders a beer.

The bartender says, "You know what this is? A double entendre."

The man looks at him and says, "No, no. This is a double entendre."

"A double entendre? Why not a triple entendre?"

"No, no, this is a double entendre."

The man says, "What do you mean, a triple entendre?"

"No, no, this is a triple entendre."

The

In [14]:
torch.save(model.state_dict(), os.path.join("gpt2_medium_joker.pt"))